### Imports: setting up our environment

In [1]:
import logging
import time

import awkward as ak
import cabinetry
import cloudpickle
import correctionlib
from coffea import processor
from coffea.analysis_tools import PackedSelection
from coffea.dataset_tools import preprocess,apply_to_fileset, max_files, max_chunks
from coffea.dataset_tools.dataset_query import DataDiscoveryCLI
import copy
import dask
from dask_jobqueue import HTCondorCluster
from dask.distributed import Client, performance_report
import datetime
import gzip
import hist
import hist.dask as hda
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
import pyhf

import utils  # contains code for bookkeeping and cosmetics, as well as some boilerplate

logging.getLogger("cabinetry").setLevel(logging.INFO)

In [2]:
### GLOBAL CONFIGURATION

XCACHE_CHOICE = "xcache05"

# input files per process, set to e.g. 10 (smaller number = faster)
N_FILES_MAX_PER_SAMPLE = None #If None, there is no max
N_CHUNKS_MAX_PER_FILE = None #If None, there is no max
# number of seconds before giving up on file reading
TIMEOUT = 300

# number of distributed workers to request
MAX_WORKERS = 250

# enable Dask
USE_DASK = True

# enable ServiceX
USE_SERVICEX = False

### ML-INFERENCE SETTINGS

# enable ML inference
USE_INFERENCE = False

# enable inference using NVIDIA Triton server
USE_TRITON = False

In [3]:
class TtbarAnalysis(processor.ProcessorABC):
    def __init__(self, use_inference, use_triton):

        # NOTE: START VERSION MIGRATION REGION (VMR), which has been migrated
        # initialize dictionary of hists for signal and control region
        self.hist_dict = {}
        for region in ["4j1b", "4j2b"]:
            self.hist_dict[region] = (
                hda.Hist(hist.axis.Regular(utils.config["global"]["NUM_BINS"], 
                                  utils.config["global"]["BIN_LOW"], 
                                  utils.config["global"]["BIN_HIGH"], 
                                  name="observable", 
                                  label="observable [GeV]"),
                    hist.axis.StrCategory([], name="process", label="Process", growth=True),
                    hist.axis.StrCategory([], name="variation", label="Systematic variation", growth=True),
                    storage=hist.storage.Weight()
                )
            )
        
        self.cset = correctionlib.CorrectionSet.from_file("corrections.json")
        self.use_inference = use_inference

        # NOTE: END VMR
        # set up attributes only needed if USE_INFERENCE=True
        if self.use_inference:
            
            # initialize dictionary of hists for ML observables
            self.ml_hist_dict = {}
            for i in range(len(utils.config["ml"]["FEATURE_NAMES"])):
                self.ml_hist_dict[utils.config["ml"]["FEATURE_NAMES"][i]] = (
                    hist.Hist.new.Reg(utils.config["global"]["NUM_BINS"],
                                      utils.config["ml"]["BIN_LOW"][i],
                                      utils.config["ml"]["BIN_HIGH"][i],
                                      name="observable",
                                      label=utils.config["ml"]["FEATURE_DESCRIPTIONS"][i])
                    .StrCat([], name="process", label="Process", growth=True)
                    .StrCat([], name="variation", label="Systematic variation", growth=True)
                    .Weight()
                )
            
            self.use_triton = use_triton

    def only_do_IO(self, events):
        for branch in utils.config["benchmarking"]["IO_BRANCHES"][
            utils.config["benchmarking"]["IO_FILE_PERCENT"]
        ]:
            if "_" in branch:
                split = branch.split("_")
                object_type = split[0]
                property_name = "_".join(split[1:])
                ak.materialized(events[object_type][property_name])
            else:
                ak.materialized(events[branch])
        return {"hist": {}}

    def process(self, events):
        if utils.config["benchmarking"]["DISABLE_PROCESSING"]:
            # IO testing with no subsequent processing
            return self.only_do_IO(events)

        process = events.metadata["metadata"]["process"]  # "ttbar" etc.
        variation = events.metadata["metadata"]["variation"]  # "nominal" etc.

        # normalization for MC
        x_sec = events.metadata["metadata"]["xsec"]
        nevts_total = ak.num(events,axis=0)
        lumi = 3378 # /pb
        if process != "data":
            xsec_weight = x_sec * lumi / nevts_total
        else:
            xsec_weight = 1

        # setup triton gRPC client
        if self.use_inference and self.use_triton:
            triton_client = utils.clients.get_triton_client(utils.config["ml"]["TRITON_URL"])


        #### systematics
        # jet energy scale / resolution systematics
        # need to adjust schema to instead use coffea add_systematic feature, especially for ServiceX
        # cannot attach pT variations to events.jet, so attach to events directly
        # and subsequently scale pT by these scale factors
        events["pt_scale_up"] = 1.03
        events["pt_res_up"] = utils.systematics.jet_pt_resolution(events.Jet.pt,events.Jet.phi)

        syst_variations = ["nominal"]
        jet_kinematic_systs = ["pt_scale_up", "pt_res_up"]
        event_systs = [f"btag_var_{i}" for i in range(4)]
        if process == "wjets":
            event_systs.append("scale_var")

        # Only do systematics for nominal samples, e.g. ttbar__nominal
        if variation == "nominal":
            syst_variations.extend(jet_kinematic_systs)
            syst_variations.extend(event_systs)

        # for pt_var in pt_variations:
        for syst_var in syst_variations:
            ### event selection
            # very very loosely based on https://arxiv.org/abs/2006.13076

            # Note: This creates new objects, distinct from those in the 'events' object
            elecs = events.Electron
            muons = events.Muon
            jets = events.Jet
            if syst_var in jet_kinematic_systs:
                # Replace jet.pt with the adjusted values
                jets["pt"] = jets.pt * events[syst_var]

            electron_reqs = (elecs.pt > 30) & (abs(elecs.eta) < 2.1) & (elecs.cutBased == 4) & (elecs.sip3d < 4)
            muon_reqs = ((muons.pt > 30) & (abs(muons.eta) < 2.1) & (muons.tightId) & (muons.sip3d < 4) &
                         (muons.pfRelIso04_all < 0.15))
            jet_reqs = (jets.pt > 30) & (abs(jets.eta) < 2.4) & (jets.isTightLeptonVeto)

            # Only keep objects that pass our requirements
            elecs = elecs[electron_reqs]
            muons = muons[muon_reqs]
            jets = jets[jet_reqs]

            if self.use_inference:
                even = (events.event%2==0)  # whether events are even/odd

            B_TAG_THRESHOLD = 0.5

            ######### Store boolean masks with PackedSelection ##########
            selections = PackedSelection(dtype='uint64')
            # Basic selection criteria
            selections.add("exactly_1l", (ak.num(elecs) + ak.num(muons)) == 1)
            selections.add("atleast_4j", ak.num(jets) >= 4)
            selections.add("exactly_1b", ak.sum(jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) == 1)
            selections.add("atleast_2b", ak.sum(jets.btagCSVV2 > B_TAG_THRESHOLD, axis=1) >= 2)
            # Complex selection criteria
            selections.add("4j1b", selections.all("exactly_1l", "atleast_4j", "exactly_1b"))
            selections.add("4j2b", selections.all("exactly_1l", "atleast_4j", "atleast_2b"))

            #for region in ["4j1b", "4j2b"]: #FIXME: DELETE LATER
            for region in ["4j2b"]:
                region_selection = selections.all(region)
                region_jets = jets[region_selection]
                region_elecs = elecs[region_selection]
                region_muons = muons[region_selection]
                region_weights = ak.ones_like(ak.num(region_jets)) * xsec_weight
                if self.use_inference:
                    region_even = even[region_selection]

                if region == "4j1b":
                    observable = ak.sum(region_jets.pt, axis=-1)

                elif region == "4j2b":

                    # reconstruct hadronic top as bjj system with largest pT
                    trijet = ak.combinations(region_jets, 3, fields=["j1", "j2", "j3"])  # trijet candidates
                    trijet["p4"] = trijet.j1 + trijet.j2 + trijet.j3  # calculate four-momentum of tri-jet system
                    trijet["max_btag"] = np.maximum(trijet.j1.btagCSVV2, np.maximum(trijet.j2.btagCSVV2, trijet.j3.btagCSVV2))
                    btag_filt = (trijet.j1.btagCSVV2 > B_TAG_THRESHOLD) | (trijet.j2.btagCSVV2 > B_TAG_THRESHOLD) | (trijet.j3.btagCSVV2 > B_TAG_THRESHOLD)
                    trijet = trijet[btag_filt]  # at least one-btag in trijet candidates
                    # pick trijet candidate with largest pT and calculate mass of system
                    trijet_mass = trijet["p4"][ak.argmax(trijet.p4.pt, axis=1, keepdims=True)].mass
                    observable = ak.flatten(trijet_mass)

                    if ak.sum(region_selection)==0:
                        continue

                    if self.use_inference:
                        features, perm_counts = utils.ml.get_features(
                            region_jets,
                            region_elecs,
                            region_muons,
                            max_n_jets=utils.config["ml"]["MAX_N_JETS"],
                        )
                        even_perm = np.repeat(region_even, perm_counts)

                        # calculate ml observable
                        if self.use_triton:
                            results = utils.ml.get_inference_results_triton(
                                features,
                                even_perm,
                                triton_client,
                                utils.config["ml"]["MODEL_NAME"],
                                utils.config["ml"]["MODEL_VERSION_EVEN"],
                                utils.config["ml"]["MODEL_VERSION_ODD"],
                            )

                        else:
                            results = utils.ml.get_inference_results_local(
                                features,
                                even_perm,
                                utils.ml.model_even,
                                utils.ml.model_odd,
                            )
                            
                        results = ak.unflatten(results, perm_counts)
                        features = ak.flatten(ak.unflatten(features, perm_counts)[
                            ak.from_regular(ak.argmax(results,axis=1)[:, np.newaxis])
                        ])
                syst_var_name = f"{syst_var}"
                # Break up the filling into event weight systematics and object variation systematics
                if syst_var in event_systs:
                    for i_dir, direction in enumerate(["up", "down"]):
                        # Should be an event weight systematic with an up/down variation
                        if syst_var.startswith("btag_var"):
                            i_jet = int(syst_var.rsplit("_",1)[-1])   # Kind of fragile
                            wgt_variation = self.cset["event_systematics"].evaluate("btag_var", direction, region_jets.pt[:,i_jet])
                        elif syst_var == "scale_var":
                            # The pt array is only used to make sure the output array has the correct shape
                            wgt_variation = self.cset["event_systematics"].evaluate("scale_var", direction, region_jets.pt[:,0])
                        syst_var_name = f"{syst_var}_{direction}"
                        self.hist_dict[region].fill(
                            observable=observable, process=process,
                            variation=syst_var_name, weight=region_weights * wgt_variation
                        )
                        if region == "4j2b" and self.use_inference:
                            for i in range(len(utils.config["ml"]["FEATURE_NAMES"])):
                                self.ml_hist_dict[utils.config["ml"]["FEATURE_NAMES"][i]].fill(
                                    observable=features[..., i], process=process,
                                    variation=syst_var_name, weight=region_weights * wgt_variation
                                )
                else:
                    # Should either be 'nominal' or an object variation systematic
                    if variation != "nominal":
                        # This is a 2-point systematic, e.g. ttbar__scaledown, ttbar__ME_var, etc.
                        syst_var_name = variation
                    self.hist_dict[region].fill(
                        observable=observable, process=process,
                        variation=syst_var_name, weight=region_weights
                    )
                    if region == "4j2b" and self.use_inference:
                        for i in range(len(utils.config["ml"]["FEATURE_NAMES"])):
                            self.ml_hist_dict[utils.config["ml"]["FEATURE_NAMES"][i]].fill(
                                observable=features[..., i], process=process,
                                variation=syst_var_name, weight=region_weights
                            )


        output = {"nevents": {events.metadata["dataset"]: ak.num(events,axis=0)}, "hist_dict": self.hist_dict}
        if self.use_inference:
            output["ml_hist_dict"] = self.ml_hist_dict

        return output

    def postprocess(self, accumulator):
        return accumulator

### "Fileset" construction and metadata

Here, we gather all the required information about the files we want to process: paths to the files and asociated metadata.

In [4]:
dataset_definition = {
    #ttbar_hadronic
    "/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAOD-106X_upgrade2018_realistic_v11_L1v1-v1/NANOAODSIM": {
        "short_name": "ttbar",
        "metadata": {"process": "ttbar",
                     "variation": "nominal",
                     "xsec": 377.96}
    },
    #Single top s-channel
    "/ST_s-channel_4f_hadronicDecays_TuneCP5_13TeV-amcatnlo-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM": {
        "short_name": "single_top_s_chan",
        "metadata": {"process": "single_top_s_chan",
                     "variation": "nominal",
                     "xsec": 7.104}
    },
    #Single top t-channel top
    "/ST_t-channel_top_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM": {
        "short_name": "single_top_t_chan_top",
        "metadata": {"process": "single_top_t_chan_top",
                     "variation": "nominal",
                     "xsec": 113.3}
    },
    #Single top t-channel antitop
    "/ST_t-channel_antitop_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM": {
        "short_name": "single_top_t_chan_antitop",
        "metadata": {"process": "single_top_t_chan_antitop",
                     "variation": "nominal",
                     "xsec": 67.91}
    },
    #Single top tW antitop
    "/ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM": {
        "short_name": "single_top_tW_antitop",
        "metadata": {"process": "single_top_tW_antitop",
                     "variation": "normal",
                     "xsec": 32.51}
    },
    #Single top tW top
    "/ST_tW_top_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v2/NANOAODSIM": {
        "short_name": "single_top_tW_top",
        "metadata": {"process": "single_top_tW_top",
                     "variation": "normal",
                     "xsec": 32.45}
    },
    #WJets 1 jet
    "/WJetsToLNu_1J_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM": {
        "short_name": "wjets_1j",
        "metadata": {"process": "wjets_1j",
                     "variation": "normal",
                     "xsec": 8832.0}
    },
    #WJets 2 jets
    "/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1/NANOAODSIM": {
        "short_name": "wjets_2j",
        "metadata": {"process": "wjets_2j",
                     "variation": "normal",
                     "xsec": 3276.0}
    }
}

In [5]:
ddc = DataDiscoveryCLI()
ddc.load_dataset_definition(dataset_definition,
                                        query_results_strategy="all",
                                        replicas_strategy="round-robin");

Processing query: 
/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAOD-106X_upgrade2018_realistic_v11_L1v1-v1/NANOAODS
IM

Output()

                                                      Query:                                                       
/TTToHadronic_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAOD-106X_upgrade2018_realistic_v11_L1v1-v1/NANOAODS
                                                        IM                                                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┓
┃ Name                                ┃ Tag                                                                      ┃┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇┩
│ TTToHadronic_TuneCP5_13TeV-powheg-… │ (1) RunIISummer20UL18NanoAOD-106X_upgrade2018_realistic_v11_L1v1-v1/NAN… ││
└─────────────────────────────────────┴──────────────────────────────────────────────────────────────────────────┴┘

Use the command select to selected the datasets

Selected datasets:

...

Sites availability for dataset: 
/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1
/NANOAODSIM

                 Available replicas                 
┏━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Index ┃ Site            ┃ Files   ┃ Availability ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━┩
│   0   │ T1_US_FNAL_Disk │ 63 / 63 │    100.0%    │
│   1   │ T2_DE_DESY      │ 63 / 63 │    100.0%    │
│   2   │ T1_DE_KIT_Disk  │ 63 / 63 │    100.0%    │
│   3   │ T2_US_Nebraska  │ 63 / 63 │    100.0%    │
│   4   │ T1_RU_JINR_Disk │ 63 / 63 │    100.0%    │
│   5   │ T3_IT_Trieste   │ 63 / 63 │    100.0%    │
│   6   │ T3_KR_UOS       │ 63 / 63 │    100.0%    │
│   7   │ T1_IT_CNAF_Disk │ 49 / 63 │    77.8%     │
│   8   │ T1_ES_PIC_Disk  │ 14 / 63 │    22.2%     │
│   9   │ T2_US_Caltech   │ 14 / 63 │    22.2%     │
└───────┴─────────────────┴─────────┴──────────────┘

Replicas for 
/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_v16_L1v1-v1
/NANOAODSIM
├── T1_DE_KIT_Disk
│   ├── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│   │   mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/32780857-517D-284B-B247-E31C2
│   │   708CC13.root
│   ├── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│   │   mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/07152B19-A70A-8B4A-A301-96EBE
│   │   1501BF5.root
│   ├── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│   │   mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/0BD074B6-C3F6-B144-A7CE-30B82
│   │   2187BF8.root
│   ├── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│   │   mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/4D2DA600-7F1F-864E-8E0E-22580
│   │   4B30B4A.root
│   ├── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│   │   mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/6130B8D6-10BC-7E43-9F84-61CC7
│   │   C958B27.root
│   ├── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│   │   mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/693DE047-68B8-D347-8964-110D3
│   │   0EBBB28.root
│   ├── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│   │   mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/CBFC5092-C418-B54A-B392-CA9E1
│   │   1A9512B.root
│   └── root://cmsdcache-kit-disk.gridka.de:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-a
│       mcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/E18B8E81-DC18-1D4A-B32C-B15DF
│       2B7015D.root
├── T1_US_FNAL_Disk
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13Te
│   │   V-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/33C4F6BF-ED9D-044E-9764-56
│   │   36387C83C5.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13Te
│   │   V-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/C8F2D011-1A2C-6544-966D-15
│   │   4C11C94618.root
│   ├── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13Te
│   │   V-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/685FB7B0-436F-0648-B579-66
│   │   09024E3BB3.root
│   └── root://cmsdcadisk.fnal.gov//dcache/uscmsdisk/store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13Te
│       V-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2550000/7B724B8E-B146-7D45-B355-30
│       2D2251035F.root
├── T3_IT_Trieste
│   ├── root://cmsxrd.ts.infn.it:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX
│   │   -pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/551AC9D0-75D3-4742-9AC1-0AD1D3A2034E.roo
│   │   t
│   ├── root://cmsxrd.ts.infn.it:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX
│   │   -pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/DAD95557-938F-D342-8DC1-977476638EA4.roo
│   │   t
│   ├── root://cmsxrd.ts.infn.it:1094//store/mc/RunIISummer20UL18NanoAODv9/WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX
│   │   -pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2540000/E93C0042-4088-414C-88E3-D5DFF0769DD1.roo
│   │   t
│   ├── root://cmsxrd.ts.in

Selected datasets:

                                                 Selected datasets                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳┳┓
┃ … ┃ Dataset                                                                                                   ┃┃┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇╇┩
│ 1 │ /TTToHadronic_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAOD-106X_upgrade2018_realistic_v11_L1v1-… │││
│ 2 │ /ST_s-channel_4f_hadronicDecays_TuneCP5_13TeV-amcatnlo-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade20… │││
│ 3 │ /ST_t-channel_top_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/RunIISummer20UL18NanoAODv9-106… │││
│ 4 │ /ST_t-channel_antitop_4f_InclusiveDecays_TuneCP5_13TeV-powheg-madspin-pythia8/RunIISummer20UL18NanoAODv9… │││
│ 5 │ /ST_tW_antitop_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade20… │││
│ 6 │ /ST_tW_top_5f_inclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_r… │││
│ 7 │ /WJetsToLNu_1J_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_… │││
│ 8 │ /WJetsToLNu_2J_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer20UL18NanoAODv9-106X_upgrade2018_realistic_… │││
└───┴───────────────────────────────────────────────────────────────────────────────────────────────────────────┴┴┘

In [6]:
os.environ["CONDOR_CONFIG"] = "/etc/condor/condor_config"

try:
    _x509_localpath = (
        [
            line
            for line in os.popen("voms-proxy-info").read().split("\n")
            if line.startswith("path")
        ][0]
        .split(":")[-1]
        .strip()
    )
except Exception as err:
    raise RuntimeError(
        "x509 proxy could not be parsed, try creating it with 'voms-proxy-init'"
    ) from err
_x509_path = f'/scratch/{os.environ["USER"]}/{_x509_localpath.split("/")[-1]}'
os.system(f"cp {_x509_localpath} {_x509_path}")
_x509_path = os.path.basename(_x509_localpath)

INITIAL_DIR = f'/scratch/{os.environ["USER"]}'
cluster = HTCondorCluster(
    cores=1,
    memory="4 GB",
    disk="1 GB",
    death_timeout = '60',
    #python="/usr/local/bin/python3.8",
    job_extra_directives={
        "+JobFlavour": '"tomorrow"',
        "log": "dask_job_output.$(PROCESS).$(CLUSTER).log",
        "output": "dask_job_output.$(PROCESS).$(CLUSTER).out",
        "error": "dask_job_output.$(PROCESS).$(CLUSTER).err",
        "should_transfer_files": "yes",
        "when_to_transfer_output": "ON_EXIT_OR_EVICT",
        "container_image": "/scratch/rsimeon/notebook.sif",
        "InitialDir": INITIAL_DIR,
        "transfer_input_files": f'{_x509_path},/scratch/rsimeon/notebook.sif'
    },
    job_script_prologue=[
        "export XRD_RUNFORKHANDLER=1",
        f"export X509_USER_PROXY={_x509_path}",
    ]
)
print('Condor logs, output files, error files in {}'.format(INITIAL_DIR))
cluster.adapt(minimum=1, maximum=MAX_WORKERS)
client = Client(cluster)
print("HTCondorCluster client created")

client

Condor logs, output files, error files in /scratch/rsimeon
HTCondorCluster client created


Connection method: Cluster object,Cluster type: dask_jobqueue.HTCondorCluster
Dashboard: proxy/8787/status,
Dashboard: proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://144.92.181.248:16529,Workers: 0
Dashboard: proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
t0 = time.monotonic()

with client:
    ddc.do_preprocess(output_file="AGC_fileset",
                                   step_size=10000,
                                   align_to_clusters=False,
                                   scheduler_url=cluster.scheduler_address)

exec_time = time.monotonic() - t0
print(f"\npreprocessing took {exec_time:.2f} seconds")

Output()

Saved available fileset chunks to AGC_fileset_available.json.gz

Saved all fileset chunks to AGC_fileset_all.json.gz


preprocessing took 223.27 seconds


In [8]:
client.shutdown()

2024-08-21 20:38:38,437 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [4]:
os.environ["CONDOR_CONFIG"] = "/etc/condor/condor_config"

try:
    _x509_localpath = (
        [
            line
            for line in os.popen("voms-proxy-info").read().split("\n")
            if line.startswith("path")
        ][0]
        .split(":")[-1]
        .strip()
    )
except Exception as err:
    raise RuntimeError(
        "x509 proxy could not be parsed, try creating it with 'voms-proxy-init'"
    ) from err
_x509_path = f'/scratch/{os.environ["USER"]}/{_x509_localpath.split("/")[-1]}'
os.system(f"cp {_x509_localpath} {_x509_path}")
_x509_path = os.path.basename(_x509_localpath)

INITIAL_DIR = f'/scratch/{os.environ["USER"]}'
cluster = HTCondorCluster(
    cores=1,
    memory="4 GB",
    disk="2 GB",
    death_timeout = '60',
    #python="/usr/local/bin/python3.8",
    job_extra_directives={
        "+JobFlavour": '"tomorrow"',
        "log": "dask_job_output.$(PROCESS).$(CLUSTER).log",
        "output": "dask_job_output.$(PROCESS).$(CLUSTER).out",
        "error": "dask_job_output.$(PROCESS).$(CLUSTER).err",
        "should_transfer_files": "yes",
        "when_to_transfer_output": "ON_EXIT_OR_EVICT",
        "container_image": "/scratch/rsimeon/notebook.sif",
        "InitialDir": INITIAL_DIR,
        "transfer_input_files": f'{_x509_path},/scratch/rsimeon/notebook.sif'
    },
    job_script_prologue=[
        "export XRD_RUNFORKHANDLER=1",
        f"export X509_USER_PROXY={_x509_path}",
    ]
)
print('Condor logs, output files, error files in {}'.format(INITIAL_DIR))
cluster.adapt(minimum=1, maximum=MAX_WORKERS)
client = Client(cluster)
print("HTCondorCluster client created")

client

Condor logs, output files, error files in /scratch/rsimeon
HTCondorCluster client created


/opt/conda/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 4789 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.HTCondorCluster
Dashboard: proxy/4789/status,
Dashboard: proxy/4789/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://144.92.181.248:3419,Workers: 0
Dashboard: proxy/4789/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
FILESET_LOC = "AGC_fileset_available.json.gz"
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
with client, performance_report(filename=f"reports/{XCACHE_CHOICE}_dask{timestamp}_{MAX_WORKERS}_{N_FILES_MAX_PER_SAMPLE}_{N_CHUNKS_MAX_PER_FILE}.html"):
    print(f"Applying to signal fileset {FILESET_LOC}")
    with gzip.open(FILESET_LOC, "rt") as file:
        fileset_full = json.load(file)
    #Only do at most first # files
    if N_FILES_MAX_PER_SAMPLE:
        fileset_maxfiles = max_files(fileset_full,N_FILES_MAX_PER_SAMPLE)
    else:
        fileset_maxfiles = fileset_full
    #Only do at most first # chunks of each file
    if N_CHUNKS_MAX_PER_FILE:
        fileset_maxchunks = max_chunks(fileset_maxfiles,N_CHUNKS_MAX_PER_FILE)
    else:
        fileset_maxchunks = fileset_maxfiles
    #Change filepaths to use Wisconsin XCache
    fileset_ready = {}
    for dset in fileset_maxchunks:
        dataset_ready = {}
        for key, val in fileset_maxchunks[dset].items():
            if key != 'files':
                dataset_ready[key] = val
            else:
                files_info = {}
                for fkey, fval in val.items():
                    fparts = fkey.split('/store')
                    if len(fparts) != 2:
                        raise ValueError(f"Filepath {fkey} in dataset {dset} does not fit pattern (splittable on '/store'")
                    xfname = '/store'.join([f'root://cms{XCACHE_CHOICE}.hep.wisc.edu/',fparts[-1]])
                    files_info[xfname] = fval
                dataset_ready[key] = files_info
        fileset_ready[dset] = dataset_ready
    print("Starting clock")
    t0 = time.monotonic()
    #Run across the fileset (if set up correctly, a lazy dask operation)
    outputs, reports = apply_to_fileset(TtbarAnalysis(USE_INFERENCE, USE_TRITON),fileset_ready,uproot_options={"allow_read_errors_with_report": True, "skipbadfiles": True, "timeout": TIMEOUT})
    #Actually compute the outputs
    print('About to compute signal outputs')
    coutputs, creports = dask.compute(outputs,reports)
    print('Finished computing signal outputs')

exec_time = time.monotonic() - t0
print(f"\nexecution with XCache took {exec_time:.2f} seconds")

Applying to signal fileset AGC_fileset_available.json.gz
Starting clock


/opt/conda/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/opt/conda/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:243: RuntimeWarning: Missing cross-reference index for FatJet_genJetAK8Idx => GenJetAK8
  warnings.warn(


About to compute signal outputs
Finished computing signal outputs

execution with XCache took 3048.88 seconds


In [6]:
client.shutdown()

2024-11-11 16:29:08,265 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [7]:
import pickle as pkl
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
with open(f"reports/{XCACHE_CHOICE}_reports{timestamp}_{MAX_WORKERS}_{N_FILES_MAX_PER_SAMPLE}_{N_CHUNKS_MAX_PER_FILE}.pkl","wb") as f:
    pkl.dump(creports,f)

In [11]:
#utils.metrics.read_reports(creports)
print("Metrics can be read by uncommenting first line in this cell")

Metrics can be read by uncommenting first line in this cell
